In [ ]:
workers = 10

worker = (1..workers).map {|x| "panworker-#{x}"}

["panworker-6", "panworker-7", "panworker-8", "panworker-9", "panworker-10"]

In [ ]:
create_worker = "aws lightsail create-instances --instance-names {#{worker.map{|x| "'#{x}'"}.join(',')}} --availability-zone 'ap-southeast-1a' --blueprint-id 'ubuntu_20_04' --bundle-id 'medium_2_0'"
puts create_worker
system(create_worker)

aws lightsail create-instances --instance-names {'panworker-1','panworker-2','panworker-3','panworker-4','panworker-5'} --availability-zone 'ap-southeast-1a' --blueprint-id 'ubuntu_20_04' --bundle-id 'medium_2_0'


In [ ]:
get_public_ip = "aws lightsail get-instances --region 'ap-southeast-1' --query 'instances[].{name:name,publicIpAddress:publicIpAddress}'"
puts get_public_ip
system(get_public_ip)

aws lightsail get-instances --region 'ap-southeast-1' --query 'instances[].{name:name,publicIpAddress:publicIpAddress}'


In [ ]:
public_ips = JSON.parse('''

[
    {
        "name": "panworker-3",
        "publicIpAddress": "54.169.245.225"
    },
    {
        "name": "panworker-2",
        "publicIpAddress": "54.169.2.13"
    },
    {
        "name": "panworker-7",
        "publicIpAddress": "13.212.208.128"
    },
    {
        "name": "panworker-4",
        "publicIpAddress": "13.212.211.5"
    },
    {
        "name": "panworker-5",
        "publicIpAddress": "13.229.217.135"
    },
    {
        "name": "panworker-6",
        "publicIpAddress": "13.229.53.237"
    },
    {
        "name": "panworker-9",
        "publicIpAddress": "13.229.111.153"
    },
    {
        "name": "panworker-8",
        "publicIpAddress": "54.255.213.21"
    },
    {
        "name": "Develop",
        "publicIpAddress": "13.250.244.138"
    },
    {
        "name": "panworker-1",
        "publicIpAddress": "13.229.51.103"
    },
    {
        "name": "panworker-10",
        "publicIpAddress": "54.179.119.166"
    },
    {
        "name": "panana",
        "publicIpAddress": "18.139.158.81"
    }
]    

''').map {|x| [x["name"],x["publicIpAddress"]]}.to_h

{"panworker-3"=>"54.169.245.225", "panworker-2"=>"54.169.2.13", "panworker-7"=>"13.212.208.128", "panworker-4"=>"13.212.211.5", "panworker-5"=>"13.229.217.135", "panworker-6"=>"13.229.53.237", "panworker-9"=>"13.229.111.153", "panworker-8"=>"54.255.213.21", "Develop"=>"13.250.244.138", "panworker-1"=>"13.229.51.103", "panworker-10"=>"54.179.119.166", "panana"=>"18.139.158.81"}

In [ ]:
def run_cmd(ip,cmd)
  puts "====begin cmd #{time=Time.now} @#{ip}===="
  system("ssh -i ~/.ssh/LightsailDefaultKey-ap-southeast-1.pem -o 'StrictHostKeyChecking no' ubuntu@#{ip} '#{cmd}'")
  puts "====end cmd #{Time.now} @#{ip} time #{Time.now-time} s===="
end



require 'parallel'

def worker_run(worker,public_ips,cmd)
  puts "====worker_run begin cmd #{time=Time.now}===="
  Parallel.map(worker,in_threads: 5) { |w| run_cmd(public_ips[w],cmd) }
  puts "====worker_run end cmd #{Time.now} time #{Time.now-time} s===="
end

def worker_run_script(worker,public_ips,script)
  cmd = script.split("\n")
  cmd.map {|c|
    worker_run(worker,public_ips,c)
  }
end

:worker_run_script

In [ ]:
script = <<~CODE
sudo apt-get update
sudo apt-get install -y ca-certificates curl gnupg lsb-release
curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /usr/share/keyrings/docker-archive-keyring.gpg
echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/docker-archive-keyring.gpg] https://download.docker.com/linux/ubuntu $(lsb_release -cs) stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
sudo apt-get update
sudo apt-get install -y docker-ce docker-ce-cli containerd.io
sudo usermod -aG docker $USER
newgrp docker 
sudo systemctl enable docker.service
sudo systemctl enable containerd.service
CODE


["sudo apt-get update"]

In [ ]:
worker_run_script(worker,public_ips,script)
worker_run_script(worker,public_ips,"docker run hello-world")

script = <<~CODE
docker container run -d -p 80:8888 --restart=always --name panana adam429/pan-repo:panana
docker logs panana >& panana.log
CODE
worker_run_script(worker,public_ips,script)

cmd = "cat panana.log"
output = Parallel.map(worker,in_threads: 5) { |w| 
  output = [w,`#{"ssh -i ~/.ssh/LightsailDefaultKey-ap-southeast-1.pem -o 'StrictHostKeyChecking no' ubuntu@#{public_ips[w]} '#{cmd}'"}`]
}

output.map {|x| 
  x[1].match /token=(.+)\n/
  
  "http://#{public_ips[x[0]]}/?token=#{$1}"
}


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ebde59c2-d67b-4e31-ad3b-1bdd07014691' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>